In [1]:
import torch
import torchvision as tv
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader


In [2]:
import pandas as pd
from PIL import Image

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cpu


In [13]:
epochs = 100
batch_size=32
learning_rate=0.001

In [3]:
df = pd.read_csv('sports/sports.csv')

In [4]:
df_train=df[df['data set']=='train']

In [15]:
df_test=df[df['data set']=='test']

In [5]:
df_train.columns

Index(['class id', 'filepaths', 'labels', 'data set'], dtype='object')

In [23]:
s = 'sports/'+df_train['filepaths'][0]
im = Image.open(s)

In [30]:
print(im.size)
im_rgb = im.convert('RGB')
print(im_rgb.size)
tens = transforms.ToTensor()
im_tensor = tens(im_rgb)
print(im_tensor)

(224, 224)
(224, 224)
<built-in method size of Tensor object at 0x000001A19FE83B50>


In [32]:
print(im_tensor.size())

torch.Size([3, 224, 224])


In [50]:
df_train[df['labels']=='high jump']

C:\Users\KIIT\AppData\Local\Temp\ipykernel_24828\713196039.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train[df['labels']=='high jump']


,class id,filepaths,labels,data set
5463,40,train/high jump/001.jpg,high jump,train
5464,40,train/high jump/002.jpg,high jump,train
5465,40,train/high jump/003.jpg,high jump,train
5466,40,train/high jump/004.jpg,high jump,train
5467,40,train/high jump/005.jpg,high jump,train
...,...,...,...,...
5617,40,train/high jump/155.jpg,high jump,train
5618,40,train/high jump/156.jpg,high jump,train
5619,40,train/high jump/157.jpg,high jump,train
5620,40,train/high jump/158.jpg,high jump,train


In [6]:
df_train_op = df_train.drop(index=5621)

In [7]:
print(df_train_op[df_train_op['labels']=='high jump'])
len(df_train_op)

      class id                filepaths     labels data set
5463        40  train/high jump/001.jpg  high jump    train
5464        40  train/high jump/002.jpg  high jump    train
5465        40  train/high jump/003.jpg  high jump    train
5466        40  train/high jump/004.jpg  high jump    train
5467        40  train/high jump/005.jpg  high jump    train
...        ...                      ...        ...      ...
5616        40  train/high jump/154.jpg  high jump    train
5617        40  train/high jump/155.jpg  high jump    train
5618        40  train/high jump/156.jpg  high jump    train
5619        40  train/high jump/157.jpg  high jump    train
5620        40  train/high jump/158.jpg  high jump    train

[158 rows x 4 columns]


13492

In [62]:
class SportsDataset(Dataset):

    def __init__(self, image_list, df_desc):
        # image_series = pd.Series(image_list)

        self.x = image_list
        self.y = list(df_desc['enc_label'])

        self.n_samples = len(df_desc)      
         

    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    
    def __len__(self):
        return self.n_samples

In [8]:
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))])
image_list=[]
for i in range(len(df_train)):
    s = 'sports/'+df_train['filepaths'][i]
    if('.jpg' not in s)and('.jpeg' not in s):
        continue
    image = Image.open(s).convert('RGB')
    image_tensor=transform(image)
    image_list.append(image_tensor)
    print(f'Processed image {i+1}')


Processed image 1
Processed image 2
Processed image 3
Processed image 4
Processed image 5
Processed image 6
Processed image 7
Processed image 8
Processed image 9
Processed image 10
Processed image 11
Processed image 12
Processed image 13
Processed image 14
Processed image 15
Processed image 16
Processed image 17
Processed image 18
Processed image 19
Processed image 20
Processed image 21
Processed image 22
Processed image 23
Processed image 24
Processed image 25
Processed image 26
Processed image 27
Processed image 28
Processed image 29
Processed image 30
Processed image 31
Processed image 32
Processed image 33
Processed image 34
Processed image 35
Processed image 36
Processed image 37
Processed image 38
Processed image 39
Processed image 40
Processed image 41
Processed image 42
Processed image 43
Processed image 44
Processed image 45
Processed image 46
Processed image 47
Processed image 48
Processed image 49
Processed image 50
Processed image 51
Processed image 52
Processed image 53
Pr

In [10]:
len(image_list)==len(df_train_op)

True

In [53]:
sports_dict={}
k=0
for i in set(df_train['labels']):
    if i in sports_dict.keys():
        continue
    else:
        sports_dict[i] = k
        k+=1

print(sports_dict)

{'canoe slamon': 0, 'surfing': 1, 'uneven bars': 2, 'pole dancing': 3, 'roller derby': 4, 'harness racing': 5, 'luge': 6, 'steer wrestling': 7, 'bike polo': 8, 'bungee jumping': 9, 'shuffleboard': 10, 'wheelchair basketball': 11, 'field hockey': 12, 'pole vault': 13, 'skydiving': 14, 'jousting': 15, 'bowling': 16, 'curling': 17, 'hydroplane racing': 18, 'bmx': 19, 'ampute football': 20, 'tug of war': 21, 'croquet': 22, 'rugby': 23, 'bobsled': 24, 'log rolling': 25, 'football': 26, 'archery': 27, 'figure skating men': 28, 'sky surfing': 29, 'judo': 30, 'pommel horse': 31, 'cricket': 32, 'hockey': 33, 'wheelchair racing': 34, 'figure skating women': 35, 'sailboat racing': 36, 'snowmobile racing': 37, 'bull riding': 38, 'rock climbing': 39, 'water polo': 40, 'arm wrestling': 41, 'formula 1 racing': 42, 'fly fishing': 43, 'balance beam': 44, 'hammer throw': 45, 'volleyball': 46, 'sidecar racing': 47, 'baseball': 48, 'speed skating': 49, 'track bicycle': 50, 'pole climbing': 51, 'horseshoe 

In [54]:
len(sports_dict)

100

In [57]:
df_train_op['enc_label'] = df_train_op['labels']
for i in range(len(df_train_op)):
    if(i==5621):
        continue
    df_train_op['enc_label'][i] = sports_dict[df_train_op['labels'][i]]

C:\Users\KIIT\AppData\Local\Temp\ipykernel_31344\1332283892.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_op['enc_label'][i] = sports_dict[df_train_op['labels'][i]]


In [61]:
df_train_op.columns

Index(['class id', 'filepaths', 'labels', 'data set', 'enc_label'], dtype='object')

In [63]:
dataset = SportsDataset(image_list, df_train_op)

In [66]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [85]:
df_test['enc_label'] = df_test['labels']
for i in range(13493,13493+len(df_test)):
    # if(i==5621):
        # continue
    df_test['enc_label'][i] = sports_dict[df_test['labels'][i]]

C:\Users\KIIT\AppData\Local\Temp\ipykernel_31344\3733320619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['enc_label'] = df_test['labels']
C:\Users\KIIT\AppData\Local\Temp\ipykernel_31344\3733320619.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['enc_label'][i] = sports_dict[df_test['labels'][i]]


In [86]:
df_test

,class id,filepaths,labels,data set,enc_label
13493,0,test/air hockey/1.jpg,air hockey,test,99
13494,0,test/air hockey/2.jpg,air hockey,test,99
13495,0,test/air hockey/3.jpg,air hockey,test,99
13496,0,test/air hockey/4.jpg,air hockey,test,99
13497,0,test/air hockey/5.jpg,air hockey,test,99
...,...,...,...,...,...
13988,99,test/wingsuit flying/1.jpg,wingsuit flying,test,72
13989,99,test/wingsuit flying/2.jpg,wingsuit flying,test,72
13990,99,test/wingsuit flying/3.jpg,wingsuit flying,test,72
13991,99,test/wingsuit flying/4.jpg,wingsuit flying,test,72


In [26]:
test_image_list=[]
banned=[]
for i in range(13493,13493+len(df_test)):
    s = 'sports/'+df_test['filepaths'][i]
    if('.jpg' not in s)and('.jpeg' not in s):
        banned.append(i)
        continue
    image = Image.open(s).convert('RGB')
    image_tensor=transform(image)
    test_image_list.append(image_tensor)
    print(f'Processed image {i+1}')

Processed image 13494
Processed image 13495
Processed image 13496
Processed image 13497
Processed image 13498
Processed image 13499
Processed image 13500
Processed image 13501
Processed image 13502
Processed image 13503
Processed image 13504
Processed image 13505
Processed image 13506
Processed image 13507
Processed image 13508
Processed image 13509
Processed image 13510
Processed image 13511
Processed image 13512
Processed image 13513
Processed image 13514
Processed image 13515
Processed image 13516
Processed image 13517
Processed image 13518
Processed image 13519
Processed image 13520
Processed image 13521
Processed image 13522
Processed image 13523
Processed image 13524
Processed image 13525
Processed image 13526
Processed image 13527
Processed image 13528
Processed image 13529
Processed image 13530
Processed image 13531
Processed image 13532
Processed image 13533
Processed image 13534
Processed image 13535
Processed image 13536
Processed image 13537
Processed image 13538
Processed 

In [87]:
test_dataset = SportsDataset(test_image_list, df_test)
# print(len(test_image_list)==len(df_test))

In [88]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [50]:
classes = tuple(df['labels'].unique())
print(classes)
print(len(classes))

('air hockey', 'ampute football', 'archery', 'arm wrestling', 'axe throwing', 'balance beam', 'barell racing', 'baseball', 'basketball', 'baton twirling', 'bike polo', 'billiards', 'bmx', 'bobsled', 'bowling', 'boxing', 'bull riding', 'bungee jumping', 'canoe slamon', 'cheerleading', 'chuckwagon racing', 'cricket', 'croquet', 'curling', 'disc golf', 'fencing', 'field hockey', 'figure skating men', 'figure skating pairs', 'figure skating women', 'fly fishing', 'football', 'formula 1 racing', 'frisbee', 'gaga', 'giant slalom', 'golf', 'hammer throw', 'hang gliding', 'harness racing', 'high jump', 'hockey', 'horse jumping', 'horse racing', 'horseshoe pitching', 'hurdles', 'hydroplane racing', 'ice climbing', 'ice yachting', 'jai alai', 'javelin', 'jousting', 'judo', 'lacrosse', 'log rolling', 'luge', 'motorcycle racing', 'mushing', 'nascar racing', 'olympic wrestling', 'parallel bar', 'pole climbing', 'pole dancing', 'pole vault', 'polo', 'pommel horse', 'rings', 'rock climbing', 'roller 

In [67]:
dataiter = iter(train_dataloader)
data = dataiter.__next__()
images, labels = data
print(data)

[tensor([[[[-0.3255, -0.2784, -0.1608,  ..., -0.3333, -0.2627, -0.2471],
          [-0.0353,  0.0196,  0.0745,  ..., -0.3255, -0.2627, -0.2471],
          [ 0.1686,  0.2000,  0.2078,  ..., -0.3569, -0.3020, -0.3020],
          ...,
          [ 0.0196,  0.0588,  0.0431,  ..., -0.1059, -0.1059,  0.0510],
          [ 0.0431,  0.0588,  0.0431,  ..., -0.0039, -0.0745, -0.0824],
          [ 0.0745,  0.0902,  0.0980,  ...,  0.0431, -0.0275, -0.0667]],

         [[-0.4039, -0.3490, -0.2314,  ..., -0.3647, -0.3098, -0.2941],
          [-0.1137, -0.0588,  0.0039,  ..., -0.3569, -0.3020, -0.2863],
          [ 0.0667,  0.1216,  0.1373,  ..., -0.3647, -0.3255, -0.3255],
          ...,
          [ 0.2941,  0.3333,  0.3098,  ...,  0.1765,  0.1765,  0.3333],
          [ 0.3098,  0.3255,  0.3098,  ...,  0.2627,  0.1922,  0.1843],
          [ 0.3412,  0.3569,  0.3647,  ...,  0.3098,  0.2392,  0.2000]],

         [[-0.4745, -0.4196, -0.3020,  ..., -0.3882, -0.3098, -0.2941],
          [-0.1843, -0.1294, 

In [68]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1= nn.Linear(128 * (224//8)*(224//8), 512)
        self.fc2 = nn.Linear(512, 100)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128*(224//8)*(224//8))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [69]:
model = ConvNet()

In [70]:
print(images.size())
op1 = model.conv1(images)
print(op1.size())
op2 = model.pool(op1)
print(op2.size())
op3 = model.conv2(op2)
print(op3.size())
op4 = model.pool(op3)
print(op4.size())
op5 = model.conv3(op4)
print(op5.size())
op6 = model.pool(op5)
print(op6.size())


torch.Size([32, 3, 224, 224])
torch.Size([32, 32, 224, 224])
torch.Size([32, 32, 112, 112])
torch.Size([32, 64, 112, 112])
torch.Size([32, 64, 56, 56])
torch.Size([32, 128, 56, 56])
torch.Size([32, 128, 28, 28])


In [71]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [72]:
n_total_steps = len(train_dataloader)
print(n_total_steps)

422


In [84]:
for epoch in range(epochs):
    try:
        for i, (images, lab) in enumerate(train_dataloader):
            # images = images.to(device)
            # labels = labels.to(device)

            ops = model(images)
            loss = criterion(ops, lab)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print(f'Epoch: {epoch + 1}/{epochs} | step: {i + 1}/{n_total_steps} | loss: {loss.item()}')

    except Exception as e:
        print(f'Error occurred during training: {e}')
        # Optionally, you can break the loop or continue to the next epoch
        continue#`continue` depending on your needs

print('Finished training!')



Epoch: 1/100 | step: 1/422 | loss: 4.606741905212402
Epoch: 1/100 | step: 2/422 | loss: 4.59805965423584
Epoch: 1/100 | step: 3/422 | loss: 4.591215133666992
Epoch: 1/100 | step: 4/422 | loss: 4.599302291870117
Epoch: 1/100 | step: 5/422 | loss: 4.600444316864014
Epoch: 1/100 | step: 6/422 | loss: 4.600710868835449
Epoch: 1/100 | step: 7/422 | loss: 4.590400695800781
Epoch: 1/100 | step: 8/422 | loss: 4.5993733406066895
Epoch: 1/100 | step: 9/422 | loss: 4.606795310974121
Epoch: 1/100 | step: 10/422 | loss: 4.592353820800781
Epoch: 1/100 | step: 11/422 | loss: 4.599428176879883
Epoch: 1/100 | step: 12/422 | loss: 4.606945991516113
Epoch: 1/100 | step: 13/422 | loss: 4.598843097686768
Epoch: 1/100 | step: 14/422 | loss: 4.599743366241455
Epoch: 1/100 | step: 15/422 | loss: 4.597687721252441
Epoch: 1/100 | step: 16/422 | loss: 4.594789505004883
Epoch: 1/100 | step: 17/422 | loss: 4.603553771972656
Epoch: 1/100 | step: 18/422 | loss: 4.610654354095459
Epoch: 1/100 | step: 19/422 | loss: 4

In [90]:
for i, (image, label) in enumerate(test_dataloader):
    print(f'data fine->{i}')

data fine->0
data fine->1
data fine->2
data fine->3
data fine->4
data fine->5
data fine->6
data fine->7
data fine->8
data fine->9
data fine->10
data fine->11
data fine->12
data fine->13
data fine->14
data fine->15


In [99]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(100)]
    n_class_samples = [0 for i in range(100)]
    k=0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        n_samples +=labels.size(0)
        n_correct +=(predicted==labels).sum().item()

        print(len(outputs)==len(labels))

        for i in range(len(outputs)):
            label = labels[i]
            pred = predicted[i]
            if(label==pred):
                n_class_correct[label] += 1
                print(f'Matched! batch:{k+1}/image: {i+1}')
            n_class_samples[label] += 1
        
        k+=1
        
    acc = 100.0 * n_correct/n_samples
    print(f'Accuracy achieved: {acc} %')
    


Matched! batch:1/image: 2
Matched! batch:1/image: 3
Matched! batch:1/image: 4
Matched! batch:1/image: 9
Matched! batch:1/image: 17
Matched! batch:1/image: 20
Matched! batch:1/image: 23
Matched! batch:1/image: 26
Matched! batch:1/image: 27
Matched! batch:1/image: 29
Matched! batch:1/image: 30
Matched! batch:1/image: 31
Matched! batch:2/image: 2
Matched! batch:2/image: 3
Matched! batch:2/image: 11
Matched! batch:2/image: 17
Matched! batch:2/image: 18
Matched! batch:2/image: 21
Matched! batch:2/image: 24
Matched! batch:2/image: 25
Matched! batch:2/image: 27
Matched! batch:2/image: 31
Matched! batch:3/image: 4
Matched! batch:3/image: 6
Matched! batch:3/image: 10
Matched! batch:3/image: 12
Matched! batch:3/image: 13
Matched! batch:3/image: 14
Matched! batch:3/image: 18
Matched! batch:3/image: 20
Matched! batch:3/image: 22
Matched! batch:3/image: 24
Matched! batch:3/image: 29
Matched! batch:3/image: 31
Matched! batch:4/image: 8
Matched! batch:4/image: 13
Matched! batch:4/image: 14
Matched! b

In [101]:
n_correct

184